# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("cities.csv")

weather_df


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Hermanus,29,ZA,1601426481,92,-34.42,19.23,46.00,3.67
1,1,Punta Arenas,40,CL,1601426170,86,-53.15,-70.92,33.80,26.40
2,2,New Norfolk,99,AU,1601426475,71,-42.78,147.06,60.01,1.01
3,3,Pevek,100,RU,1601426551,74,69.70,170.31,39.45,19.26
4,4,Marawi,9,PH,1601426557,79,8.00,124.29,78.01,3.00
...,...,...,...,...,...,...,...,...,...,...
521,521,Malazgirt,0,TR,1601426789,68,39.14,42.54,50.16,2.84
522,522,Sitka,90,US,1601426479,81,57.05,-135.33,53.60,8.05
523,523,Yelovo,7,RU,1601426789,85,57.05,54.92,48.47,1.05
524,524,Nichinan,75,JP,1601426790,88,31.60,131.37,69.80,5.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_df["Humidity"]


In [5]:


# Create  Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 70 and 80
vacation_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 5) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()

vacation_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
51,51,Gwadar,0,PK,1601426632,78,25.12,62.33,79.90,1.50
324,324,Luzhou,0,CN,1601426722,74,28.87,105.44,70.86,1.90
395,395,Makkah al Mukarramah,0,SA,1601426745,66,21.43,39.83,79.70,3.02
451,451,Baghdad,0,IQ,1601426592,29,33.34,44.40,75.20,4.70
460,460,Nāthdwāra,0,IN,1601426767,58,24.93,73.82,73.13,3.60


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = vacation_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
51,Gwadar,PK,25.12,62.33,
324,Luzhou,CN,28.87,105.44,
395,Makkah al Mukarramah,SA,21.43,39.83,
451,Baghdad,IQ,33.34,44.40,
460,Nāthdwāra,IN,24.93,73.82,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # build url and make API call
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # get results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
     # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    

Retrieving Results for Index 51: Gwadar.
Closest hotel in Gwadar is Zaver Pearl Continental Hotel.
Retrieving Results for Index 324: Luzhou.
Closest hotel in Luzhou is Jiucheng Hotel.
Retrieving Results for Index 395: Makkah al Mukarramah.
Closest hotel in Makkah al Mukarramah is Hotel Zamazem Al Rawda.
Retrieving Results for Index 451: Baghdad.
Closest hotel in Baghdad is Baghdad Hotel.
Retrieving Results for Index 460: Nāthdwāra.
Closest hotel in Nāthdwāra is श्री Vallabh Vilas Lords Plaza.


In [10]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
51,Gwadar,PK,25.12,62.33,Zaver Pearl Continental Hotel
324,Luzhou,CN,28.87,105.44,Jiucheng Hotel
395,Makkah al Mukarramah,SA,21.43,39.83,Hotel Zamazem Al Rawda
451,Baghdad,IQ,33.34,44.40,Baghdad Hotel
460,Nāthdwāra,IN,24.93,73.82,श्री Vallabh Vilas Lords Plaza


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info =[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
 

# Add marker layer and info box content on top of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Figure
fig



Figure(layout=FigureLayout(height='420px'))